---

# CSCI 3202, Fall 2022
# Final Project
# Project Due: Thursday December 8, 2022 at 6:00 PM
## Proposals Due: Friday November 18, 2022 at 6:00 PM


You have two options for completing your final project for this course. 

#### Option 1 ####
The first option is presented in this notebook and involves implementing a Connect Four game with AB pruning and A* as player strategies. 

#### Option 2 ####
The second option is to design your own project that includes any of the algorithms we've discussed throughout the semester, or an algorithm that you're interested in learning that we haven't discussed in class. Your project also needs to include some kind of analysis of how it performed on a specific problem. If you're interested in the design your own project option, you need to discuss your idea with one of the course instructors to get approval. If you do a project without getting approval, you will receive a 0 regardless of the quality of the project. 

**The rules:**

1. Choose EITHER the given problem to submit OR choose your own project topic. 

2. If you choose your own project topic, please adhere to the following guidelines:
- Send an email to the course instructors before Friday, November 18 at 6pm, with a paragraph description of your project. We will respond within 24 hours with feedback.
- The project can include an algorithm we've discussed throughout the semester or an algorithm that you're been curious to learn. Please don't recycle a project that you did in another class. 
- If you do your own project without prior approval, you will receive a 0 for this project.
- Your project code, explanation, and results must all be contained in a Jupyter notebook. 

3. All work, code and analysis must be **your own**.
4. You may use your course notes, posted lecture slides, textbook, in-class notebooks and homework solutions as resources.  You may also search online for answers to general knowledge questions, like the form of a probability distribution function, or how to perform a particular operation in Python. You may not use entire segments of code as solutions to any part of this project, e.g. if you find a Python implementation of policy iteration online, you can't use it.
5. You may **not** post to message boards or other online resources asking for help.
6. **You may not collaborate with classmates or anyone else.**
7. This is meant to be like a coding portion of an exam. So, we will be much less helpful than we typically are with homework. For example, we will not check answers, help debug your code, and so on.
8. If you have a question, post it first as a **private** Piazza message. If we decide that it is appropriate for the entire class, then we will make it a public post (and anonymous).
9. If any part of the given project or your personal project is left open-ended, it is because we intend for you to code it up however you want. Our primary concern is with the plots/analysis that your code produces. Feel free to ask clarifying questions though.

Violation of these rules will result in an **F** and a trip to the Honor Code council.

---
**By writing your name below, you agree to abide by these rules:**

**Your name:** Maya Douglas

---


---

Some useful packages and libraries:



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import deque
import heapq
import unittest
from scipy import stats
import copy as cp
from time import time
import math
from numpy import inf

---

## Problem 1: Game Theory - Playing "intelligent" Connect Four

Connect Four is a two-player game where the objective is to get four pieces in a row - horizontally, vertically, or diagonally. Check out this video if you're unfamiliar with the game: https://www.youtube.com/watch?v=utXzIFEVPjA.

### (1a)   Defining the Connect Four class structure

We've provided the humble beginnings of a Connect Four game. You need to fill in this class structure for Connect Four using what we did during class as a guide, and then implement min-max search with AB pruning, and A* search with at least one heuristic function. The class structure includes the following:

* `moves` is a list of columns to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`. Note that when a 'move' is made, the column must have an open slot and the piece must drop to the lowest row. 
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` returns the utility of the current state if the player is Red and $-1 \cdot$ utility if the player is Black.
* `display(self)` is a method to display the current game `state`. You get it for free because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if Red wins, 0 if draw, -1 if Black wins). `player1` and `player2` are functional arguments that we will deal with in parts **1b** and **1d**.

Some notes:
* Assume Red always goes first.
* Do **not** hard-code for 6x7 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [2]:
class State:
    def __init__(self, nrow, ncol):
        self.last_move = 'B'
        self.to_move = 'R'
        self.utility = 0
        self.board = {}
        # self.moves = moves
        self.nrow = nrow
        self.ncol = ncol
        self.updateMoves()
        self.lastMovePos = (-1,-1)
    # def u(self, state):
    #     return [(x, y) for (x, y) in state.moves
    #             if x == self.h or (x + 1 , y ) in state.board]
    def updateMoves(self): 
        self.moves = [x for x in range(self.ncol) if not (self.nrow - 1, x) in self.board]

class ConnectFour:
    def __init__(self, nrow=6, ncol=7, nwin=4):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        # moves = [(row,col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)]
        self.state = State(nrow, ncol)

    def result(self, state, move):
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (R or B)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (X or O)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''
        if move not in state.moves:
            # self.display()
            # raise Exception("Move " + move + " was attemped, not legal.")
            return -1000
        
        new_state = cp.deepcopy(state)
        
        
        for i in range(self.nrow-2, -2, -1): #loops backwards itereates by -1
            if i < 0 or (i, move) in state.board:
                new_state.board[(i + 1, move)] = state.to_move 
                new_state.lastMovePos = (i + 1, move)
                break
        # Return a copy of the updated state:
        #   compute utility, update the board, remove the move, update whose turn
        new_state.updateMoves()
        new_state.last_move = state.to_move
        new_state.to_move = state.last_move
        return new_state
        
    def compute_utility(self, state, move = None):
        if move is not None:
            state = self.result(state, move)
        for player in ["R", "B"]:
            row = state.lastMovePos[0]
            col = state.lastMovePos[1]
            board = state.board

            in_a_row = 0
            for c in range(self.ncol):
                if board.get((row,c))==player:
                    in_a_row += 1
                elif in_a_row < self.nwin:
                    in_a_row = 0

            in_a_col = 0
            for r in range(self.nrow):
                if board.get((r,col))==player:
                    in_a_col += 1
                elif in_a_col < self.nwin:
                    in_a_col = 0

            in_a_diag1 = 0
            for r in range(self.nrow):
                if board.get((r,col-(row-r)))==player:
                    in_a_diag1 += 1
                elif in_a_diag1 < self.nwin: 
                    in_a_diag1 = 0
                    
            in_a_diag2 = 0
            for r in range(self.nrow):
                if board.get((r,col+(row-r)))==player:
                    in_a_diag2 += 1
                elif in_a_diag2 < self.nwin: 
                    in_a_diag2 = 0
            won = in_a_row >= self.nwin or in_a_col >= self.nwin or in_a_diag1 >= self.nwin or in_a_diag2 >= self.nwin
            if won:
                state.utility = 1 if self.state.last_move == "R" else -1
                return 1 if self.state.last_move == "R" else -1
        state.utility = 0
        return 0


    def game_over(self, state):
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''
        return len(state.moves) == 0 or self.compute_utility(state) != 0
    
    def utility(self, state, player):
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''
        
        # your code goes here...
        return state.utility if player=='R' else -state.utility
        print(state.utility)

    def display(self, state):
        print("")
        board = state.board
        for row in range(self.nrow - 1, -1, -1):
            for col in range(self.ncol):
                print(board.get((row, col), '.'), end=' ')
            print()

    def play_game(self, player1, player2):
        '''Play a game of Connect Four!'''
        
        # your code goes here...
        turn_limit = self.nrow*self.ncol  # limit in case of buggy code
        turn = 0
        while turn<=turn_limit:
            for player in [player1, player2]:
                turn += 1
                move = player(self)
                self.state = self.result(self.state, move)
                if self.game_over(self.state):
                    return self.state.utility
        return 0        

### (1b) Define a random player

Define a function `random_player` that takes a single argument of the `ConnectFour` class and returns a random move out of the available legal moves in the `state` of the `ConnectFour` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [3]:
def random_player(game):
    '''A player that chooses a legal move at random out of all
    available legal moves in ConnectFour state argument'''
    
    # your code goes here...
    possible_actions = game.state.moves
    return possible_actions[np.random.randint(low=0, high=len(possible_actions))]
    

We know from experience and/or because I'm telling you right now that if two `random_player`s play many games of ConnectFour against one another, whoever goes first will win about 55% of the time.  Verify that this is the case by playing at least 1,000 games between two random players. Report the proportion of the games that the first player has won.**(Chris: is this true for TicTacToe, or Connect Four)**

**"Unit tests":** If you are wondering how close is close enough to 55%, I simulated 100 tournaments of 1,000 games each. The min-max range of win percentage by the first player was 52-59%.

In [4]:
niter = 1000
wins = 0
draws = 0
losses = 0
#the larger niter is the worse it gets
#why
for k in range(niter):
    ttt = ConnectFour(6,7,4)
    out = ttt.play_game(random_player, random_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
print('First-player winning percentage = {}'.format(wins/niter))
print('Second-player winning percentage = {}'.format(losses/niter))
print('Draw percentage = {}'.format(draws/niter))

First-player winning percentage = 0.524
Second-player winning percentage = 0.471
Draw percentage = 0.005


### (1c) What about playing randomly on different-sized boards?

What does the long-term win percentage appear to be for the first player in a 10x10 ConnectFour tournament, where 4 marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **1b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*

In [5]:
# 1000 games between two random players
niter = 1000
wins = 0
draws = 0
losses = 0
#the larger niter is the worse it gets
#why
for k in range(niter):
    ttt = ConnectFour(10,10,6)
    out = ttt.play_game(random_player, random_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
print('First-player winning percentage = {}'.format(wins/niter))
print('Second-player winning percentage = {}'.format(losses/niter))
print('Draw percentage = {}'.format(draws/niter))

First-player winning percentage = 0.458
Second-player winning percentage = 0.45
Draw percentage = 0.092


The win percentage decreases and draw percentage increases. This is because the game tends to be more equal when it's 10 by 10 because of a balanced state space. The first-player winning and loss percentages are more equal in this case too. Bias towards the first player becomes less and less evident as state space becomes larger because it has time to balance out - the more it gets played the ledss who played first affects winning percentage. The even playing field also contributes to draws - again, less bias.

### (1d) Define an alpha-beta player

Alright. Let's finally get serious about our Connect Four game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `ConnectFour` class object and returns the minimax move in the `state` of the `ConnectFour` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

In [6]:
def alpha_beta_player(game):
    # Add variables to keep track of the minimax value and pruning percentage
    minimax_value = None
    nodes_evaluated = 0
    nodes_pruned = 0

    state = game.state
    player = game.state.to_move
    # Functions used by alpha_beta
    def max_value(state, alpha, beta, depth = 0):
        nonlocal minimax_value, nodes_evaluated, nodes_pruned

        if depth == -1000 or game.game_over(state):
            minimax_value = game.utility(state, player)
            return minimax_value
        v = -np.inf    
        for a in state.moves:
            nodes_evaluated += 1
            v = max(v, min_value(game.result(state, a), alpha, beta, depth + 1))
            if v >= beta:
                nodes_pruned += 1
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth = 0):
        nonlocal minimax_value, nodes_evaluated, nodes_pruned

        if depth == -1000 or game.game_over(state):
            minimax_value = game.utility(state, player)
            return minimax_value
        v = np.inf
        for a in state.moves:
            nodes_evaluated += 1
            v = min(v, max_value(game.result(state, a), alpha, beta, depth + 1))
            if v <= alpha:
                nodes_pruned += 1
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_search:
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.state.moves:
        v = min_value(game.result(state, a), best_score, beta)
        if v > best_score:
            best_score = v
            best_action = a
        if v >= beta:
            break

    # Calculate the pruning percentage and print the results
    # pruning_percentage = nodes_pruned / nodes_evaluated * 100
    # print("Minimax value:", minimax_value)
    # print("Nodes evaluated:", nodes_evaluated)
    # print("Nodes pruned:", nodes_pruned)
    # print("Pruning percentage:", pruning_percentage)
    return best_action


Verify that your alpha-beta player code is working appropriately through the following tests, using a standard 6x7 ConnectFour board. Run **10 games for each test**, and track the number of wins, draws and losses. Report these results for each case.

1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. Two alpha-beta players should always draw. One player is the max player and the other player is the min player.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. You are welcome to run more than 10 tests if you'd like. 

In [7]:
# 1000 games between two random players
niter = 1000
wins = 0
draws = 0
losses = 0
#the larger niter is the worse it gets
#why
for k in range(niter):
    ttt = ConnectFour(3,4,3)
    out = ttt.play_game(alpha_beta_player, random_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
print('First-player winning percentage = {}'.format(wins/niter))
print('Second-player winning percentage = {}'.format(losses/niter))
print('Draw percentage = {}'.format(draws/niter))

First-player winning percentage = 0.842
Second-player winning percentage = 0.124
Draw percentage = 0.034


### (1e) What has pruning ever done for us?

Calculate the number of number of states expanded by the minimax algorithm, **with and without pruning**, to determine the minimax decision from the initial 6x7 ConnectFour board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Then compute the percent savings that you get by using alpha-beta pruning. i.e. Compute $\frac{\text{Number of nodes expanded with pruning}}{\text{Number of nodes expanded with minimax}} $

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

I made an alpha beta pruning function that prints out the computations.

In [8]:
def alpha_beta_pruning(game):
    # Add variables to keep track of the minimax value and pruning percentage
    minimax_value = None
    nodes_evaluated = 0
    nodes_pruned = 0

    state = game.state
    player = game.state.to_move
    # Functions used by alpha_beta
    def max_value(state, alpha, beta, depth = 0):
        nonlocal minimax_value, nodes_evaluated, nodes_pruned

        if depth == -1000 or game.game_over(state):
            minimax_value = game.utility(state, player)
            return minimax_value
        v = -np.inf    
        for a in state.moves:
            nodes_evaluated += 1
            v = max(v, min_value(game.result(state, a), alpha, beta, depth + 1))
            if v >= beta:
                nodes_pruned += 1
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth = 0):
        nonlocal minimax_value, nodes_evaluated, nodes_pruned

        if depth == -1000 or game.game_over(state):
            minimax_value = game.utility(state, player)
            return minimax_value
        v = np.inf
        for a in state.moves:
            nodes_evaluated += 1
            v = min(v, max_value(game.result(state, a), alpha, beta, depth + 1))
            if v <= alpha:
                nodes_pruned += 1
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_search:
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.state.moves:
        v = min_value(game.result(state, a), best_score, beta)
        if v > best_score:
            best_score = v
            best_action = a
        if v >= beta:
            break

    # Calculate the pruning percentage and print the results
    pruning_percentage = nodes_pruned / nodes_evaluated * 100
    print("Minimax value:", minimax_value)
    print("Nodes evaluated:", nodes_evaluated)
    print("Nodes pruned:", nodes_pruned)
    print("Pruning percentage:", pruning_percentage)
    return best_action


In [9]:
game = ConnectFour(3, 4, 3)
print(alpha_beta_pruning(game))

Minimax value: -1
Nodes evaluated: 801
Nodes pruned: 295
Pruning percentage: 36.82896379525593
0


### (2) A* Search

In Part II of this project, you need to implement a player strategy to employ A* Search in order to win at ConnectFour. To test your A* player, play 10 games against the random player and 10 games against the AB pruning player. 

Write an explanation of this strategy's implementation and performance in comparison to the random player and the AB Pruning player from **1d**.

A lot of the code that you wrote for the minimax player and the Connect Four game structure can be reused for the A* player. However, you will need to write a new utility function for A* that considers the path cost and heuristic cost.


### (2a) Define a heuristic function
Your A* player will need to use a heuristic function. You have two options for heurstics: research an existing heuristic for Connect Four, or games similar to Connect Four, and use that. Or, design your own heuristic. Write a one-paragraph description of the heuristic you're using, including a citation if you used an existing heuristic.

Three-in-a-row heuristic might be a rule or strategy that a player uses to try to get three of their pieces in a row on the game board. This could involve making specific moves in certain situations, or using a particular pattern of moves to try to block the opponent's pieces and create opportunities to get three in a row. This type of heuristic's goal in AI is to lead to a better outcome than randomizing it.

### (2b) Compare A* to other algorithms
Next, play 10 games of Connect Four using your A* player and a random player and 10 games against the AB pruning player. In four or five paragraphs, report on the outcome. Did one player win more than the other? How often was the game a draw? How many moves did each player make? Were there situations where one player appeared to do better than the other? Given the outcome, are there other heuristics you would like to implement?

In [42]:
def three_in_a_row(game):
    # Count the number of three-in-a-row patterns for the given player
    state = game.state
    player = game.state.to_move
    count = 0
    for row in range(state.nrow):
        for col in range(state.ncol):
            if state.board.get((row,col)) == player:
                # Check for horizontal three-in-a-row patterns
                if col <= state.ncol - 3:
                    if state.board.get((row, col+1)) == player and state.board.get((row, col+2)) == player:
                        count += 1
                # Check for vertical three-in-a-row patterns
                if row <= state.nrow - 3:
                    if state.board.get((row+1, col)) == player and state.board.get((row+2, col)) == player:
                        count += 1
                # Check for diagonal three-in-a-row patterns
                if row <= state.nrow - 3:
                    if state.board.get((row+1, col+1)) == player and state.board.get((row+2, col+2)) == player:
                        count += 1
                        
                if row > 3:
                    if state.board.get((row-1, col+1)) == player and state.board.get((row-2, col+2)) == player:
                        count += 1
    return count

In [43]:
def astar(game):
    best = 0
    state = game.state
    player = game.state.to_move
    best_action = game.state.moves
    best_action_number = 0
    for a in game.state.moves:
        v = three_in_a_row(game)
        if v > best_action_number:
            best_action_number = v
            best_action = a
    # This retuns a number instead of an action
    return a

First we will do it against random player.

In [47]:
niter = 10
wins = 0
draws = 0
losses = 0
#
for k in range(niter):
    ttt = ConnectFour(3,4,3)
    out = ttt.play_game(astar, random_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
print('First-player winning percentage = {}'.format(wins/niter))
print('Second-player winning percentage = {}'.format(losses/niter))
print('Draw percentage = {}'.format(draws/niter))

First-player winning percentage = 1.0
Second-player winning percentage = 0.0
Draw percentage = 0.0


Then we'll make it go against alpha beta.

In [48]:
niter = 10
wins = 0
draws = 0
losses = 0
#
for k in range(niter):
    ttt = ConnectFour(3,4,3)
    out = ttt.play_game(astar, alpha_beta_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
print('First-player winning percentage = {}'.format(wins/niter))
print('Second-player winning percentage = {}'.format(losses/niter))
print('Draw percentage = {}'.format(draws/niter))

First-player winning percentage = 1.0
Second-player winning percentage = 0.0
Draw percentage = 0.0


Did one player win more than the other? My astar function usually won over alpha beta and random players, with a few exceptions. 
How often was the game a draw? It was never a draw for astar VS alpha beta and random.
How many moves did each player make? It depends on the game. 
Were there situations where one player appeared to do better than the other? I think that for the most part as long as the player was a star it did better than the other players. This is because the three-in-a-row heuristic is extremely optimal and only chooses the best path. The fact that my a star function only went with the path that had the most three in a rows, made it a much more 'intelligent' algorithm than ab pruning and random player. It also eliminated the need to have a frontier queue since the variable kept track of cost. 
Given the outcome, are there other heuristics you would like to implement? Based on my results I think I would not want to implement any other heuristic, unless I added a acase that required it to check if there's a coin in between the three. That being said more often than not having three in a row (without being consecutive) is plenty enough information. I do think that Euclidean and Manhattan distances were both covered in lecture and might let me 'feel out' how A* works instead of trusting three_in_a_row to know what it's doing, so I might practice that in my free time. However outcome wise I believe my current a star is stellar. It also expands to fewest nodes, as stated in the slides. 